In [1]:
using Pkg
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10


In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels

#### using HighDimMixedModels

In [3]:
using HighDimMixedModels
#using highDimMM

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [4]:
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

,period,herd,incid,hsz
,Float64,String,Int8,Int8
1,1.0,H01,2,14
2,2.0,H01,3,12
3,3.0,H01,4,9
4,4.0,H01,0,5
5,1.0,H02,3,22


In [5]:
#f = @formula(period ~ 0 + herd + incid + hsz)
#f = @formula(period ~ 0 +(1|herd) + incid + hsz )
f = @formula(period ~ 0 + incid + hsz + (1|herd))
contrasts = Dict( :incid => ContinuousTerm, :hsz => ContinuousTerm, :herd => CategoricalTerm)
#fit(MixedModel, @formula(period ~ incid + hsz +(1|herd)), df, REML = true)

Dict{Symbol, UnionAll} with 3 entries:
  :hsz   => ContinuousTerm
  :herd  => CategoricalTerm
  :incid => ContinuousTerm

In [6]:
f

FormulaTerm
Response:
  period(unknown)
Predictors:
  0
  incid(unknown)
  hsz(unknown)
  (herd)->1 | herd

**try for apply_schema**

In [ ]:
LinearMixedModel<:MixedModel

In [ ]:
highDimMixedModel<:MixedModel

In [ ]:
# try for f2
f2 = @formula(period ~ 0  +(1|herd) + incid + hsz)
#f2 = @formula(period ~ 0  + incid + hsz +(1|herd) )

In [ ]:
#sch = schema(f2,df,contrasts)
sch = schema(f2,df)

In [ ]:
form = apply_schema(f2, sch)

In [ ]:
form = apply_schema(f2, sch, LinearMixedModel)  ## do not need MixedModels prefix due to multiple dispatch

In [ ]:
form = apply_schema(f2, sch, highDimMixedModel)

In [ ]:
modelcols(form,df)

### Construct object

#### 3.variable names indicates M,X,Z

In [ ]:
HMM = highDimMixedModel(f, df, contrasts, "incid", "hsz", "herd")

In [ ]:
first(HMM.M.M,5)

In [ ]:
first(HMM.X.X,5)

In [ ]:
HMM.Z.Z

#### 4. parse random effect in formula

In [7]:
HMM = highDimMixedModel(f, df, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ HighDimMixedModels /Users/zyxu/.julia/dev/HighDimMixedModels/src/bricks.jl:12


In [15]:
first(HMM.M.M,10)

10-element Vector{Float64}:
 2.0
 3.0
 4.0
 0.0
 3.0
 1.0
 1.0
 8.0
 2.0
 0.0

In [9]:
first(HMM.X.X,5)

5-element Vector{Float64}:
 14.0
 12.0
  9.0
  5.0
 22.0

In [10]:
size(HMM.Z.Z)

(56, 15)

In [11]:
sch = schema(df)
form = apply_schema(f, sch, highDimMixedModel)

FormulaTerm
Response:
  period(continuous)
Predictors:
  0
  incid(continuous)
  hsz(continuous)
  (1 | herd)

In [12]:
# when the random effect is the last term, can use form directly instead of form.rhs
FixedEffectMatrix,Z = modelmatrix(form,df)

([2.0 14.0; 3.0 12.0; … ; 1.0 15.0; 0.0 15.0], [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 1.0])

In [12]:
modelmatrix(form.rhs,df)

([2.0 14.0; 3.0 12.0; … ; 1.0 15.0; 0.0 15.0], [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 1.0])

In [13]:
y, pre = modelcols(form,df)

([1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0, 1.0, 2.0, 3.0  …  3.0, 4.0, 1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0, 4.0], ([2.0 14.0; 3.0 12.0; … ; 1.0 15.0; 0.0 15.0], [1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 1.0]))

In [14]:
numOfHDM = 1
M = highDimMat(FixedEffectMatrix[:,1:numOfHDM])
X = XMat(FixedEffectMatrix[:, (numOfHDM + 1):size(FixedEffectMatrix,2)])

┌ Warning: n >= p in high dimensional matrix
└ @ HighDimMixedModels /Users/zyxu/.julia/dev/HighDimMixedModels/src/bricks.jl:12


XMat{Float64, Matrix{Float64}}([14.0; 12.0; … ; 15.0; 15.0])

In [ ]:
ml = fit(MixedModel, @formula(period ~ 0 + incid + hsz +(1|herd)), df, REML = true)

In [ ]:
ml = MixedModels.fit(MixedModel, @formula(period ~ 0 + incid + hsz +(1|herd)), df, REML = true)

In [ ]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

In [ ]:
HMM::highDimMixedModel

In [ ]:
HMM::MixedModel

In [ ]:
HMM::StatsModels.RegressionModel

In [ ]:
ml::LinearMixedModel

In [ ]:
ml::MixedModel

In [ ]:
ml::StatsModels.RegressionModel

In [ ]:
HMM::MixedModel

In [ ]:
HMM::StatsModels.RegressionModel

In [ ]:
highDimMixedModel::MixedModel

In [ ]:
HMM

In [ ]:
HMM.formula

### Now try to use apply_schema in MixedModels

In [ ]:
# try for f2
f2 = @formula(period ~ 0 + incid + hsz +(1|herd))
sch = schema(f2,df)
form = apply_schema(f2, sch, highDimMixedModel) ## multiple dispatch

In [ ]:
y,X = modelcols(form,df)

In [ ]:
Z = X[2]

In [ ]:
X[1]

In [ ]:
df.herd

In [ ]:
form.rhs

### Optimization

In [15]:
ml = fit(MixedModel, @formula(period ~ 0 + incid + hsz +(1|herd)), df, REML = true)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /Users/zyxu/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
Minimizing 16 	 Time: 0:00:00 (30.18 ms/it)


|             |    Est. |     SE |     z |      p | σ_herd |
|:----------- | -------:| ------:| -----:| ------:| ------:|
| incid       | -0.2939 | 0.0890 | -3.30 | 0.0010 |        |
| hsz         |  0.1317 | 0.0239 |  5.52 | <1e-07 |        |
| (Intercept) |         |        |       |        | 1.1569 |
| Residual    |  1.3496 |        |       |        |        |


In [16]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

sigma = [1.0, 1.0]
negLog = -57.1099018030012
The initial object value is -57.1099018030012
sigma = [1.0, 1.0]
negLog = -57.1099018030012
OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = -57.1099018030012
sigma = [1.75, 1.0]
negLog = -55.97912076973617
sigma = [1.0, 1.75]
negLog = -52.37720093978967
sigma = [0.25, 1.0]
negLog = -64.50473446075652
sigma = [1.0, 0.25]
negLog = -128.22481961167495
sigma = [0.29289321881345254, 0.0]
negLog = Inf
sigma = [0.9469669914110089, 0.19696699141100893]
negLog = -156.76581599673838
sigma = [0.995676667220638, 0.13993745974082217]
negLog = -207.31013755591323
sigma = [0.9952996645647639, 0.06493840728682465]
negLog = -405.77399467791014
sigma = [0.9878315127277438, 0.06424796674686986]
negLog = -410.13655848976134
sigma = [0.987341243774038, 0.05676400814509819]
negLog = -458.32682425399554
sigma = [0.9948025184365511, 0.05600283772361585]
negLog = -463.4181543729141
sigma = [0.9947292842170775, 0.05525642178789495]
negLog = -469.0572593

In [17]:
opt

|                          |                    |
|:------------------------ |:------------------ |
| **Initialization**       |                    |
| Initial parameter vector | [1.0, 1.0]         |
| Initial objective value  | -57.1099018030012  |
| **Optimizer settings**   |                    |
| Optimizer (from NLopt)   | `LN_BOBYQA`        |
| Lower bounds             | [0.0, 0.0]         |
| `ftol_rel`               | 1.0e-12            |
| `ftol_abs`               | 1.0e-8             |
| `xtol_rel`               | 1.0e-5             |
| `xtol_abs`               | [1.0e-10, 1.0e-10] |
| `initial_step`           | [0.75, 0.75]       |
| `maxfeval`               | -1                 |
| `maxtime`                | -1.0               |
| **Result**               |                    |
| Function evaluations     | 22                 |
| Final parameter vector   | [0.9947, 0.054]    |
| Final objective value    | -479.0182          |
| Return code              | `XTOL_REACHED`     |
